In [50]:
import pandas as pd
import numpy as np
import torch
from onehotencoder import OneHotEncoder

In [55]:
n_gram = 4
endecode = OneHotEncoder()
def get_compound_token(s, n=n_gram):
    if not isinstance(s, str) or not s or n <= 0:
        return ""

    token_parts = []
    current_length = 0
    string_index = 0

    while current_length < n and string_index < len(s):
        if s[string_index:].startswith('Cl'):
            token_parts.append('Cl')
            current_length += 1
            string_index += 2
        elif s[string_index:].startswith('Br'):
            token_parts.append('Br')
            current_length += 1
            string_index += 2
        else:
            token_parts.append(s[string_index])
            current_length += 1
            string_index += 1

    return "".join(token_parts)


In [56]:
if n_gram == 1:
    current_n_gram = endecode.encode('[BOS]')
else:
    string_series = pd.read_csv('data/train.csv', header=None)[0]
    string_series = string_series[string_series.apply(lambda x: isinstance(x,str) and x !='')]
    top_n_grams = string_series.apply(lambda s: get_compound_token(s, n=n_gram-1))
    top_chars = (top_n_grams.value_counts()/sum(top_n_grams.value_counts())).to_dict()
    token = np.random.choice(list(top_chars.keys()),p=list(top_chars.values()))
    start_token = endecode.encode('[BOS]')
    current_n_gram = endecode.encode_sequence(token,skip_append=True)
    current_n_gram = torch.tensor(np.concatenate((start_token,current_n_gram),axis=0))

print(current_n_gram)

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])
